In [ ]:
from geopy.distance import geodesic
import random

# Definir el punto fijo en el centro de la Ciudad de México
origin = (19.432608, -99.133209)  # Coordenadas aproximadas del Zócalo

# Generar un punto aleatorio a una distancia de 2 km del punto fijo
def random_point_2km_away(center, distance_km):
    angle = random.uniform(0, 360)
    destination = geodesic(kilometers=distance_km).destination(center, angle)
    return (destination.latitude, destination.longitude)

destination = random_point_2km_away(origin, 2)

print("Punto fijo (centro de la Ciudad de México):", origin)
print("Punto aleatorio a 2 km:", destination)

In [ ]:
import osmnx as ox
import geopandas as gpd

# Cargar la red de vías desde el archivo cache_MexicoCity_walk.graphml
graph = ox.load_graphml('cache_MexicoCity_walk.graphml')

# Cargar los datos de crímenes desde el archivo crimenes.geojson
crimenes = gpd.read_file('crimes.geojson')

# Mostrar información básica de los datos cargados
print(graph)
print(crimenes.head())

In [ ]:
# Re-project geometries to a projected CRS (e.g., UTM zone 14N for Mexico City)
crimenes = crimenes.to_crs(epsg=32614)

# Crear buffers de 50 metros alrededor de cada punto
crimenes['buffer'] = crimenes.geometry.buffer(50)

# Mostrar los primeros registros con los buffers
print(crimenes[['weight', 'buffer']].head())

In [ ]:
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import numpy as np

# Create a base map centered on Mexico City
map_center = [19.38, -99.15]  # Approximate center of Mexico City
crime_map = folium.Map(location=map_center, zoom_start=12, tiles='OpenStreetMap')

# Create a colormap for gradient colors
cmap = plt.cm.RdYlBu_r  # Red-Yellow-Blue reversed colormap

# Create a copy of the crimenes dataframe and convert it to WGS84 (EPSG:4326)
crimenes_wgs84 = crimenes.copy()
crimenes_wgs84 = crimenes_wgs84.to_crs(epsg=4326)

# Convert the buffer geometries to WGS84
buffer_gdf = gpd.GeoDataFrame(crimenes[['weight']], geometry=crimenes['buffer'], crs=crimenes.crs)
buffer_gdf = buffer_gdf.to_crs(epsg=4326)

# Add crime buffers to the map
for idx, row in buffer_gdf.iterrows():
    # Create a popup with crime information
    popup_text = f"""
    <b>Weight:</b> {crimenes.iloc[idx]['weight']}<br>
    """

    # Use the weight directly as the color value (0-1 range)
    weight_value = crimenes.iloc[idx]['weight']
    rgba_color = cmap(weight_value)
    hex_color = '#{:02x}{:02x}{:02x}'.format(
        int(rgba_color[0]*255),
        int(rgba_color[1]*255),
        int(rgba_color[2]*255)
    )

    # Add the buffer as a GeoJson polygon with more transparency
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x, color=hex_color: {
            'fillColor': color,
            'color': color,
            'weight': 1,
            'fillOpacity': 0.15  # Even more transparent (was 0.25)
        },
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(crime_map)

# Add a custom legend with gradient colors
gradient_legend = """
<div style="position: fixed; 
     bottom: 50px; right: 50px; width: 180px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white; padding: 10px;
     ">
     <b>Crime Weight (0 - 1)</b><br>
     <div style="background: linear-gradient(to right, blue, yellow, red); 
          height: 20px; margin-top: 5px; margin-bottom: 5px;">
     </div>
     <span style="float: left;">Low</span>
     <span style="float: right;">High</span>
     <div style="clear: both;"></div>
</div>
"""

crime_map.get_root().html.add_child(folium.Element(gradient_legend))

# Add title
title_html = '''
<h3 align="center" style="font-size:16px"><b>Crime Buffer Zones Map</b></h3>
'''
crime_map.get_root().html.add_child(folium.Element(title_html))


In [ ]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import osmnx as ox
from shapely.geometry import Polygon

display(buffer_gdf.head())

def mid_point(origin, destination):
        # Calculate the L2 distance between origin and destination
    l2_distance = np.linalg.norm(np.array(origin) - np.array(destination))
    # Calculate the midpoint between origin and destination
    midpoint = ((origin[0] + destination[0]) / 2, (origin[1] + destination[1]) / 2)

    midpoint_point = Point(midpoint[1], midpoint[0]) 
    midpoint_geom = gpd.GeoSeries([midpoint_point], crs=buffer_gdf.crs)
    buffer_radius = l2_distance

    midpoint_buffer = midpoint_geom.buffer(buffer_radius)

    return midpoint_buffer

def crop_buffers(origin, destination, buffer_gdf):

    midpoint_buffer = mid_point(origin, destination)

    filtered_buffers = buffer_gdf[buffer_gdf.intersects(midpoint_buffer.unary_union)]

    return filtered_buffers

def crop_graph(origin, destination, graph):

    midpoint_buffer = mid_point(origin, destination)

    buffer_polygon = midpoint_buffer.geometry[0]

    subgraph = ox.truncate.truncate_graph_polygon(
        graph, 
        buffer_polygon, 
        truncate_by_edge=True)
    
    return subgraph

filtered_buffers = crop_buffers(origin, destination, buffer_gdf)
filtered_graph = crop_graph(origin, destination, graph)

In [ ]:
from shapely.geometry import Point
import geopandas as gpd
import networkx as nx

# Versión vectorizada (más rápida para grafos grandes)
def label_nodes_vectorized(graph, buffer_gdf, weight_col='weight'):
    nodes = {node: Point(data['x'], data['y']) for node, data in graph.nodes(data=True)}
    gdf_nodes = gpd.GeoDataFrame(geometry=list(nodes.values()), index=nodes.keys(), crs=buffer_gdf.crs)
    
    # Spatial join
    joined = gpd.sjoin(gdf_nodes, buffer_gdf, how='left')
    node_weights = joined.groupby(joined.index)[weight_col].sum()
    
    nx.set_node_attributes(graph, node_weights.to_dict(), name='buffer_weight')
    return graph

def label_nodes_with_buffer_weights(graph, buffer_gdf, weight_col='weight', attribute_name='buffer_weight'):
    """
    Etiqueta nodos del grafo con la suma de pesos de buffers que los intersectan
    
    Args:
        graph (nx.Graph): Grafo de NetworkX
        buffer_gdf (gpd.GeoDataFrame): GeoDataFrame con buffers y columna de peso
        weight_col (str): Nombre de la columna con pesos en buffer_gdf
        attribute_name (str): Nombre del atributo a añadir a los nodos
        
    Returns:
        nx.Graph: Grafo con atributos actualizados
    """
    # Crear índice espacial para los buffers
    sindex = buffer_gdf.sindex
    
    # Convertir nodos a GeoDataFrame
    nodes = {node: (data['x'], data['y']) for node, data in graph.nodes(data=True)}
    gdf_nodes = gpd.GeoDataFrame(
        geometry=[Point(x, y) for x, y in nodes.values()],
        index=nodes.keys(),
        crs=buffer_gdf.crs
    )
    
    # Para cada nodo, encontrar buffers que lo intersectan
    for node_id, node_point in gdf_nodes.geometry.items():
        # Búsqueda espacial con índice
        possible_matches_index = list(sindex.intersection(node_point.bounds))
        possible_matches = buffer_gdf.iloc[possible_matches_index]
        precise_matches = possible_matches[possible_matches.intersects(node_point)]
        
        # Sumar pesos de buffers intersectados
        total_weight = precise_matches[weight_col].sum()
        
        # Asignar atributo al nodo
        graph.nodes[node_id][attribute_name] = total_weight
    
    return graph

# Aplicar la función a tu grafo filtrado
graph_labeled = label_nodes_vectorized(filtered_graph, filtered_buffers, weight_col='weight')

# Opcional: Convertir a GeoDataFrame para visualización
#nodes_gdf, edges_gdf = ox.graph_to_gdfs(graph_labeled)
# Mapa de calor de pesos
nc = ox.plot.get_node_colors_by_attr(graph_labeled, 'buffer_weight', cmap='viridis')
fig, ax = ox.plot_graph(graph_labeled, node_color=nc, node_size=20)

In [ ]:
from folium.plugins import HeatMap

# Create a base map centered on Mexico City
map_center = [19.38, -99.15]  # Approximate center of Mexico City
crime_heatmap = folium.Map(location=map_center, zoom_start=12, tiles='OpenStreetMap')

# Convert filtered_buffers to WGS84 if not already
if filtered_buffers.crs != 'EPSG:4326':
    points_wgs84 = filtered_buffers.to_crs(epsg=4326)
else:
    points_wgs84 = filtered_buffers

# Extract coordinates and weights for the heatmap
heatmap_data = []
for idx, row in points_wgs84.iterrows():
    # Get centroid of the polygon
    centroid = row.geometry.centroid
    lat = centroid.y
    lon = centroid.x
    weight = row['weight']  # Use the weight column for intensity
    heatmap_data.append([lat, lon, weight])

# Add the heatmap layer to the map
HeatMap(
    heatmap_data,
    radius=15,  # Adjust radius as needed
    max_zoom=13,
    gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 1: 'red'},
    blur=10  # Adjust blur as needed
).add_to(crime_heatmap)

# Add title
title_html = '''
<h3 align="center" style="font-size:16px"><b>Crime Heatmap</b></h3>
'''
crime_heatmap.get_root().html.add_child(folium.Element(title_html))

# Display the map
crime_heatmap


In [ ]:
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import numpy as np

# Create a base map centered on Mexico City
map_center = [19.38, -99.15]  # Approximate center of Mexico City
crime_map = folium.Map(location=map_center, zoom_start=12, tiles='OpenStreetMap')

# Create a colormap for gradient colors
cmap = plt.cm.RdYlBu_r  # Red-Yellow-Blue reversed colormap

# Create a copy of the crimenes dataframe and convert it to WGS84 (EPSG:4326)
crimenes_wgs84 = filtered_buffers.copy()
crimenes_wgs84 = crimenes_wgs84.to_crs(epsg=4326)

# Convert the buffer geometries to WGS84
buffer_gdf = gpd.GeoDataFrame(crimenes[['weight']], geometry=crimenes['buffer'], crs=crimenes.crs)
buffer_gdf = buffer_gdf.to_crs(epsg=4326)

# Add crime buffers to the map
for idx, row in buffer_gdf.iterrows():
    # Create a popup with crime information
    popup_text = f"""
    <b>Weight:</b> {crimenes.iloc[idx]['weight']}<br>
    """

    # Use the weight directly as the color value (0-1 range)
    weight_value = crimenes.iloc[idx]['weight']
    rgba_color = cmap(weight_value)
    hex_color = '#{:02x}{:02x}{:02x}'.format(
        int(rgba_color[0]*255),
        int(rgba_color[1]*255),
        int(rgba_color[2]*255)
    )

    # Add the buffer as a GeoJson polygon with more transparency
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x, color=hex_color: {
            'fillColor': color,
            'color': color,
            'weight': 1,
            'fillOpacity': 0.15  # Even more transparent (was 0.25)
        },
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(crime_map)

# Add a custom legend with gradient colors
gradient_legend = """
<div style="position: fixed; 
     bottom: 50px; right: 50px; width: 180px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white; padding: 10px;
     ">
     <b>Crime Weight (0 - 1)</b><br>
     <div style="background: linear-gradient(to right, blue, yellow, red); 
          height: 20px; margin-top: 5px; margin-bottom: 5px;">
     </div>
     <span style="float: left;">Low</span>
     <span style="float: right;">High</span>
     <div style="clear: both;"></div>
</div>
"""

crime_map.get_root().html.add_child(folium.Element(gradient_legend))

# Add title
title_html = '''
<h3 align="center" style="font-size:16px"><b>Crime Buffer Zones Map</b></h3>
'''
crime_map.get_root().html.add_child(folium.Element(title_html))

crime_map


In [ ]:
# Encontrar los nodos más cercanos al origen y destino
origin_node = ox.distance.nearest_nodes(graph, origin[1], origin[0])
destination_node = ox.distance.nearest_nodes(graph, destination[1], destination[0])

# Calcular la ruta más corta
shortest_route = ox.shortest_path(graph, origin_node, destination_node, weight='length')

# Extraer las coordenadas de la ruta
route_coords = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in shortest_route]

# Añadir la ruta al mapa con un color más llamativo y mayor grosor
folium.PolyLine(route_coords, color='red', weight=8, opacity=0.8).add_to(crime_map)

# Mostrar el mapa con la ruta
#crime_map